In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

import os

In [2]:
all_synthetic_trj = pd.read_csv('data_save/all_synthetic_trj.csv', header=0)
traj_info = pd.read_csv('data_save/all_state_change_point_record.csv', header=0)
results = pd.read_csv('data_save/scale_sensitivity_analysis_results.csv', header = 0)

In [3]:
def impacts_of_acc_plot(all_mae, state_change_label, jerk_tar, acc_array):
    
    index_space = int(jerk_tar/0.5)-1
    
    folder_path = "figure_save_acc/"
    
    if os.path.isdir(folder_path) == False:
        os.makedirs(folder_path)
    
    
    for i in [0, 2, 3, 4]:
    #for i in [0]:
    
        data = all_mae[i]
        
        state = state_change_label[i]
        title = 'state_'+state+'_jerk_tar_'+str(jerk_tar)
        plt.title(title)
        
        plot_save_path = './figure_save_acc/'+'state_'+state+'_jerk_tar_'+str(jerk_tar)+'.png'
        
        for j in [0, 1, 2, 3, 4]:
            part = data[(data.index -index_space)/5 %5 == j]
            #print('part\n', part)
            plt.plot(list(part.mean(axis=0)), label='acc_'+str(acc_array[j]))
        plt.ylabel('mae(s)',fontsize=16)
        plt.xlabel('scale',fontsize=16)
        plt.legend()
        plt.savefig(plot_save_path, dpi=600)
        plt.close('all')
        
        plot_save_path = './figure_save_acc/'+'state_'+state+'_jerk_tar_'+str(jerk_tar)+'_zoom.png'
        plt.title(title)
        for j in [0, 1, 2, 3, 4]:
            part = data[(data.index -index_space)/5 %5 == j]
            #print('part\n', part)
            plt.plot(list(part.mean(axis=0)), label='acc_'+str(acc_array[j]))
        plt.ylabel('mae(s)',fontsize=16)
        plt.xlabel('scale',fontsize=16)
        plt.xlim(0, 64)
        plt.ylim(0, 2)
        plt.legend()
        plt.savefig(plot_save_path, dpi=600)
        plt.close('all')

In [4]:
def impacts_of_acc(filter_condition, jerk_tar, results):
    
    state_change_label = {0: 'A-F', 1: 'A-D', 2: 'S-A', 3: 'F-D', 4: 'D-S'}
    mae_columns = [c for c in results.columns if 'mae' in c]
    
    all_mae = {}
    mean_mae = {}
    sub_result_dict = {}
    
    acc_array = [1.0, 1.5, 2.0, 2.5, 3.0]
     
    for look_at_label in [0, 2, 3, 4]:    

        sub_results = results[results['state_change_label']  == look_at_label]
        sub_result_dict[look_at_label] = sub_results

        if filter_condition:
            
            filter_results = sub_results
            #filter_results = sub_results[sub_results['test_jerk'] == jerk_tar]

            filter_results = filter_results[filter_results['test_jerk'].astype('float') == jerk_tar]

        else:
            filter_results = sub_results
        
        all_mae[look_at_label] = filter_results[mae_columns]

        #mean_mae[look_at_label] = list(all_mae[look_at_label].mean(axis=0))

    impacts_of_acc_plot(all_mae, state_change_label, jerk_tar, acc_array)
    
    return all_mae

In [5]:
for j in [0.5, 1.0, 1.5, 2.0, 2.5]:
    all_mae = impacts_of_acc(True, j, results)